In [1]:
import json

In [2]:
file_path = "../data/training_data.json"

with open(file_path, "r") as file:
    data = json.load(file)
    
# print(data)

In [3]:
data[0]["predictions"][0]["result"][:3]

[{'value': {'start': 449, 'end': 452, 'labels': ['NEG']},
  'id': 'ent0',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'},
 {'value': {'start': 452, 'end': 468, 'labels': ['NSCO']},
  'id': 'ent1',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'},
 {'value': {'start': 844, 'end': 856, 'labels': ['NSCO']},
  'id': 'ent2',
  'from_name': 'label',
  'to_name': 'text',
  'type': 'labels'}]

### LIST OF NEGATION WORD AND SPECULATION WORD

In [25]:
"""
def extract_words_from_indices(text, word_indices):
    extracted_words = []
    for start, end in word_indices:
        extracted_words.append(text[start:end])
    return extracted_words

"""
# Example text
# text = "This is a sample text for demonstration purposes."

# Example word indices (start and end positions of each word)
# word_indices = [(0, 4), (5, 7), (8, 9), (10, 16), (17, 21), (22, 25), (26, 39), (40, 48)]

# Extract words from text using indices
# extracted_words = extract_words_from_indices(text, word_indices)

# print(extracted_words)


In [113]:
def extract_words_from_indices(text, word_indices):
    extracted_words = []
    for (start, end), nega in word_indices:
        word_ap = text[start:end]
        word_ap = re.sub(r'[^a-zA-Z ]', '', word_ap)
        # Remove trailing spaces
        word_ap = word_ap.rstrip()
        # Remove leading spaces
        word_ap = word_ap.lstrip()
        word_ap = str(word_ap) + "\t\t" + str(nega)
        # print(word_ap)
        extracted_words.append(word_ap)
    return extracted_words

# Example text
# text = "This is a sample text for demonstration purposes."

# Example word indices (start and end positions of each word)
# word_indices = [(0, 4), (5, 7), (8, 9), (10, 16), (17, 21), (22, 25), (26, 39), (40, 48)]

# Extract words from text using indices
# extracted_words = extract_words_from_indices(text, word_indices)

# print(extracted_words)


In [114]:
import re
nega_word = []
spec_word = []
for inst in data:
    # print(inst["data"]["text"])
    text = inst["data"]["text"]
    # print("-"*10)
    nega_temp = []
    spec_temp = []
    for index, j in enumerate(inst["predictions"][0]["result"]):
        # print(j)
        # print(j["value"]["labels"])
        if(j["value"]["labels"] == ["NEG"]):
            negation = "[PREN]"
            for k in range(index+1,len(inst["predictions"][0]["result"])):
                if(inst["predictions"][0]["result"][k]["value"]["labels"] != ["NSCO"]):
                    break
                if(inst["predictions"][0]["result"][k]["value"]["start"] <  j["value"]["start"] and negation == "[PREN]"):
                    negation = "[POST]"
                #elif(inst["predictions"][0]["result"][k]["value"]["start"] >  j["value"]["start"] and negation == "[POST]"):
                #    print("Error: we have scope before and after")
                #    aa = (j["value"]["start"], j["value"]["end"])
                #    bb = inst["predictions"][0]["result"]
                #    print(f"{aa} and index is {bb}")
                    
            nega_temp.append(((j["value"]["start"], j["value"]["end"]),negation))
        elif(j["value"]["labels"] == ["UNC"]):
            spec_temp.append(((j["value"]["start"], j["value"]["end"]),"PSEU"))
    # print(nega_temp)
    # print(spec_temp)
    ne_word = extract_words_from_indices(text, nega_temp)
    #print(ne_word)
    nega_word.extend(ne_word)
    spec_word.extend(extract_words_from_indices(text, spec_temp))
    
nega_word = set(nega_word)
spec_word = set(spec_word)
print(nega_word)
print(spec_word)

{'niega\t\t[PREN]', 'incapacidad para\t\t[PREN]', 'negaitvo\t\t[PREN]', 'descartada\t\t[PREN]', 'afebril\t\t[POST]', 'ex\t\t[PREN]', 'negatiu\t\t[PREN]', 'exfumador\t\t[PREN]', 'negatividad\t\t[PREN]', 'sense\t\t[PREN]', 'inespecificos\t\t[PREN]', 'no\t\t[PREN]', 'retirar\t\t[PREN]', 'desaparicion de\t\t[PREN]', 'afebril\t\t[PREN]', 'sin\t\t[PREN]', 'negatividad del\t\t[PREN]', 'inespecifico\t\t[PREN]', 'negativas\t\t[PREN]', 'descarta\t\t[POST]', 'desorientado\t\t[PREN]', 'irregulares\t\t[POST]', 'negativo\t\t[PREN]', 'imposibilidad de\t\t[PREN]', 'negativa\t\t[PREN]', 'sin\t\t[POST]', 'falta de\t\t[PREN]', 'no\t\t[POST]', 'negativos\t\t[POST]', 'descarta\t\t[PREN]', 'suspendido\t\t[PREN]', 'ausencia de\t\t[POST]', 'se suspende\t\t[PREN]', 'se desestimo\t\t[PREN]', 'neg\t\t[PREN]', 'se retira\t\t[PREN]', 'ceden\t\t[PREN]', 'niegan\t\t[PREN]', 'en ninguna\t\t[PREN]', 'tampoco\t\t[PREN]', 'negatividad de\t\t[PREN]', 'inestabilidad\t\t[PREN]', 'indetectable\t\t[PREN]', 'retiro\t\t[PREN]'

In [13]:
def write_list_to_file(word_list, filename):
    with open(filename, 'w') as file:
        for word in word_list:
            file.write(word + '\n')

In [116]:
# Name of the file to write to
filename = '../data/negation_word.txt'

# Call the function to write the list to the file
write_list_to_file(nega_word, filename)

In [117]:
# Name of the file to write to
filename = '../data/speculation_word.txt'

# Call the function to write the list to the file
write_list_to_file(spec_word, filename)

### LIST OF MEDICAL TERMS

In [3]:
from preprocessing import *
from collections import Counter

In [4]:
ROOT_DIR = os.path.dirname(os.path.abspath(""))
train_data = load_tokens(os.path.join(ROOT_DIR, "data", "training_data_tokens.json"))

In [9]:
words = Counter()

for d in range(len(data)):
	doc = data[d]["predictions"][0]["result"]
	for s in range(len(doc)):
		sent = doc[s]["value"]
		if sent["labels"] in [["NEG"], ["UNC"]]:
			continue
		start = sent["start"]
		end = sent["end"]
		context_words, _ = get_tokens(train_data[d], start, end)
		words.update(context_words)

In [18]:
# sorted by frequency
words_lst = sorted(words.items(), key=lambda x: x[1], reverse=True)
words_lst = [word[0] for word in words_lst]
words_lst = [word for word in words_lst if len(word) > 3] # heuristic to remove short words (medial terms are long)

In [19]:
filename = '../data/medical_terms.txt'
write_list_to_file(words_lst, filename)